In [1]:
!/opt/bin/nvidia-smi

Fri Feb  3 15:09:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    32W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


load dataset

In [4]:
import re
import pandas as pd
import numpy as np
import tqdm
import nltk
import csv
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [5]:
def filter(docs):
    n = len(docs)
    bek = []
    for i in range(n):
        if "\n" == docs[i]:
            bek.append(i)

    docs = [docs[i] for i in range(n) if (i not in bek)]
    return docs

In [74]:
#Load data
dataset_path = '/content/drive/MyDrive/Notebooks/data/'
with open(dataset_path + 'SwedishCorpus.txt') as f:
    lines = f.readlines()

docs = filter(lines)

# Creating a python list of URLs:
with open(dataset_path + 'articlelinks.csv', newline='') as f:
    reader = csv.reader(f)
    article_urls = list(reader)
    article_urls.reverse() # original order was from the newest

# Trimming the list of articles to reduce computation time in further steps:
url_list = []
date = []
for coronaday in article_urls:
    for newsarticle in coronaday[1:]:
        if 'lokalt' not in newsarticle: # excluding articles with local (regional) news
            url_list.append(newsarticle)
            date.append(coronaday[0])
            
url_list = url_list[:2251] # Only for one year - until 17th Jan 2021
date = date[:2251]
len(date)

2251

In [75]:
def specification_date(date:list):
    date_time = []
    for time in date:
        time = time.split(" ")
        if time[1] == "januari":
            time[1] = 'Jan'
        elif time[1] == "februari":
            time[1] = 'Feb'
        elif time[1] == "mars":
            time[1] = 'Mar'
        elif time[1] == "april":
            time[1] = 'Apr'
        elif time[1] == "maj":
            time[1] = 'May'
        elif time[1] == "juni":
            time[1] = 'Jun'
        elif time[1] == "juli":
            time[1] = 'Jul'
        elif time[1] == "augusti":
            time[1] = 'Aug'
        elif time[1] == "september":
            time[1] = 'Sep'
        elif time[1] == "oktober":
            time[1] = 'Oct'
        elif time[1] == "november":
            time[1] = 'Nov'
        elif time[1] == "december":
            time[1] = 'Dec' 
        time = '-'.join(time)
        date_time.append(time)
    return date_time

date = specification_date(date)

In [91]:
date_2021 = []
for time in date[2212:]:
    year = '2021'
    time = '-'.join([time,year])
    date_2021.append(time)

In [95]:
date = date[:2212] + date_2021

In [96]:
len(date) 

2251

In [97]:
date = pd.to_datetime(date, format = '%d-%b-%Y')

In [9]:
# Stop Words:
stop = stopwords.words('swedish') 

# Expanding by a list of 150 most common words in this corpus (meaningful words manually deleted) (.csv from SketchEngine):
with open(dataset_path + 'stopwords.csv', newline ='',encoding ='utf-8') as f:
    reader = csv.reader(f)
    extrastops = list(reader)
    
for ex in extrastops:
    stop.append(ex[0])
    
stop_words = set(stop)

In [10]:
# Tokenization
tokenizer = nltk.RegexpTokenizer(r"\w+") # deleting punctuation

tokenized_corpus = []
for article in docs:
    # Deleting a standard phrase about SVT policy, which occurs in the end of every article:
    article = article.replace("SVT:s nyheter ska stå för saklighet och opartiskhet. Det vi publicerar ska vara sant och relevant. Vid akuta nyhetslägen kan det vara svårt att få alla fakta bekräftade, då ska vi berätta vad vi vet – och inte vet. Läs mer","")
    article = article.lower()
    article = tokenizer.tokenize(article)
    article = [word for word in article if word not in stop_words and not word.isdigit()]
    tokenized_corpus.append(article)

corpus = tokenized_corpus

In [11]:
#Joining back the articles to str to comply with the initial input type for Stanza Model:
docs = []
for article in corpus:
    out_str = " "
    docs.append(out_str.join(article))

In [24]:
len(docs)

2251

bertopic-unsup

In [12]:
!pip install bertopic
from bertopic import BERTopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 KB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 72.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 662.4/662.4 KB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer

umap_model = UMAP(n_neighbors=10, n_components=10, min_dist=0.0, metric='cosine') # UMAP is an technique for dimensionality reduction

#cluster_model = KMeans(n_clusters=7)
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True) # HDBSCAN used to cluster our documents into clusters of similar documents
vectorizer_model = CountVectorizer(ngram_range=(1, 1), stop_words="english") # Using the CountVectorizer to extract all possible words 

In [14]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("all-mpnet-base-v2")
embeddings = sentence_model.encode(docs)

In [15]:
# The n_gram_range parameter relates to the number of words that we want in our topic representation. 
# For example, "New" and "York" are two seperate words but are often used as "New York" which represents an n-gram of 2. 
# The top_n_words refers to the number of words per topic that we want extracted.
# The min_topic_size is used to specify what the minimum size of a topic can be. The lower this value the more topics are created. 
# The nr_topics specifies, after training the topic model, the number of topics that will be reduced to.
# The calculate_probabilities calculate the probabilities of each topic to each document.

topic_model = BERTopic(embedding_model=sentence_model, umap_model=umap_model, hdbscan_model=hdbscan_model, vectorizer_model=vectorizer_model, 
                       calculate_probabilities= True, low_memory= False,n_gram_range = (1, 1), nr_topics="auto")

topics, probs = topic_model.fit_transform(docs, embeddings)

In [16]:
freq = topic_model.get_topic_info(); freq.head(20)

Topic  Count                                              Name
0      -1    805           -1_personer_sverige_covid_coronaviruset
1       0    260               0_regeringen_sverige_svenska_länder
2       1    173                1_viruset_kina_coronaviruset_wuhan
3       2    127                      2_patienter_covid_iva_vården
4       3    125  3_munskydd_folkhälsomyndigheten_kommuner_tegnell
5       4    106                   4_vaccin_vaccinet_fas_vaccinera
6       5     89                      5_usa_trump_donald_president
7       6     42                     6_league_spelas_lagen_matcher
8       7     36         7_restauranger_hotell_hotellet_restaurang
9       8     36             8_norwegian_sas_flygbolag_flygbolaget
10      9     35                     9_iran_turkiet_teheran_israel
11     10     34                    10_eu_länder_kommissionen_euro
12     11     33        11_konserter_föreställningar_skjuts_ställs
13     12     32        12_johnson_boris_cummings_premiärministern
14     13     30                         13_sas_ud_resenärer_flyga
15     14     30                14_testa_tester_testas_antikroppar
16     15     27          15_antikroppar_immun_immunitet_infektion
17     16     21                              16_msb_appen_sms_app
18     17     18      17_wennergren_alexandra_herrera_kristiansson
19     18     18       18_bolsonaro_brasilien_president_brasiliens

In [17]:
topic_model.visualize_documents(docs,hide_annotations=True)

In [18]:
fig = topic_model.visualize_topics(); fig

In [98]:
topics_over_time = topic_model.topics_over_time(docs=docs, 
                                                timestamps=date, 
                                                global_tuning=True, 
                                                evolution_tuning=True
                                                )

In [ ]:
docs[0]

'coronaviruset tagit människoliv utanför kina upptäckts oron växer inför kinas största resehelg miljard kineser beräknas resande fot staden smittan först utbröt upprättat karantän flygplats tågstationen feber resa coronaviruset ursprungligen upptäcktes kina upptäckts thailand japan hittills personer dött smittats kinesiska nyåret antågande växer oron lungviruset sprida utvecklas liknar sarsepidemi tog människoliv år sen ulrika bergsten plats peking myndigheterna vidtagit omfattande åtgärder begränsa eventuell smittspridning lärt viruset fiskemarknanden stängd resa wuhan uppvisar sjukdomstecken lokaliserat personer kontakt smittade länge sett sjukdomen smittar människor viss oro viruset mutera wallensten biträdande statsepidemiolog anser risken smittspridning nuläget verkar låg nej hittills handlat personer smittade kina sen rest länderna kunnat spridits vidare länderna morgonstudion sarsepidemin kom kina mörkat information verkar förändrats wallensten annorlunda arbete delar informatio

In [99]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)